In [44]:
import random

class Neuron:
    def __init__(self, numberOfOutgoing, lst = None):
        self.activation = random.uniform(0, 1)
        self.bias = random.uniform(0, 1)
        if lst is None:
            lst = []
            for i in range(numberOfOutgoing):
                lst.append(random.uniform(0, 1))
        self.outgoingWeights = lst

    def cout(self):
        print("Activation: ", self.activation)
        print("Bias: ", self.bias)
        print("Outgoing Weights: ", self.outgoingWeights)


In [45]:
n = Neuron(10)
n.cout()

Activation:  0.6155314237420193
Bias:  0.012239498618272338
Outgoing Weights:  [0.6680992738080916, 0.06643450095017522, 0.8785752540780151, 0.3088469342479203, 0.9559097124672936, 0.8952774000831684, 0.9619843355122459, 0.2813387858361315, 0.4091931974986702, 0.3495999184315287]
